In [10]:
import pandas as pd
import numpy as np
import db_connections
from datetime import datetime, timedelta, date
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200
import warnings
warnings.filterwarnings('ignore')

In [55]:
ups_map = pd.read_csv('ups_map.csv', encoding='latin', sep=';')
ups_map_cdt_eaps = ups_map[(ups_map['CODI_EP']==208) & (
                            ups_map['CODI_TIPUS_TIT']==3) & (
                            ups_map['CODI_GRUP_UP']==20) & (
                            ups_map['CODI_SUBGRUP_UP']==21) & (
                            ups_map['CODI_REGIO']==62)][['CODI_UP','NOM_COMPLET_UP','LATITUD','LONGITUD','COORDENADA_X','COORDENADA_Y']].drop_duplicates(keep='last').reset_index(drop=True)

In [56]:
ups_map_cdt_eaps

,CODI_UP,NOM_COMPLET_UP,LATITUD,LONGITUD,COORDENADA_X,COORDENADA_Y
0,40,EAP Les Borges del Camp,"41,172869307","1,020996752","333993,95","4559835,285"
1,41,EAP Constantí,"41,154093509","1,215356799","350255,585","4557398,13"
2,42,EAP Falset,"41,147483153",",823334842","317341,81","4557412,88"
3,44,EAP Montblanc,"41,373440911","1,165822702","346613,457","4581836,949"
4,45,EAP Mont-Roig del Camp,"41,084825315",",957787068","328462,128","4550183,05"
5,47,EAP Reus 2 - Sant Pere,"41,150121257","1,092572416","339942,926","4557175,66"
6,48,EAP Reus 4 - Horts de Miró,"41,153752883","1,114659217","341805,111","4557538,463"
7,50,EAP Tarragona 1 - Bonavista-La Canonja,"41,118992214","1,191521617","348174,607","4553542,48"
8,51,EAP Tarragona 2 - La Granja-Torreforta,"41,122836852","1,214171648","350084,959","4553930,071"
9,52,EAP Tarragona 5 - Sant Pere i Sant Pau,"41,1391814","1,255984415","353631,511","4555673,492"


In [5]:
def ecap_pat(dbl):

    q = f"""SELECT 
                USUA_CIP AS CIP,
                USUA_PATNUMBER AS NHC,
        		TO_DATE(USUA_DATA_NAIXEMENT,'J') AS DATA_NAIXEMENT,
                TO_DATE(USUA_DATA_ACT,'J') AS USUA_DATA_ACT,
        		USUA_SEXE AS SEXE,
        		USUA_UAB_UP AS UP,
        		USUA_UAB_CODI_UAB AS UAB,
        		USUA_CODI_INF AS UI,
        		SUBSTR(GMA_COD, 3, 1) AS GMA,
                GMA_PLNG,
                '{dbl}' AS ETL_SOURCE,
                '{datetime.now()}' AS ETL_TIME
        	FROM 
        		USUTB040{dbl}
            LEFT JOIN
                GMATB002{dbl}
            ON
                USUTB040.USUA_CIP=GMATB002.GMA_CIP
        	WHERE 
        		USUTB040.USUA_SITUACIO = 'A'
        		AND USUTB040.USUA_UAB_UP IS NOT NULL
        		AND USUTB040.USUA_DATA_BAIXA = 1"""
    
    df = db_connections.oracle2pd('ecap', q)

    return df

def ecap_uab(dbl):

    q = f"""SELECT 
        		UAB_CODI_UP AS UP,
        		UAB_CODI_UAB AS UAB,
        		UAB_DESCRIPCIO AS METGE,
                '{dbl}' AS ETL_SOURCE,
                '{datetime.now()}' AS ETL_TIME
        	FROM 
        		VISTB039{dbl}
        	WHERE 
        		UAB_CODI_UP IN (SELECT USUA_UAB_UP FROM USUTB040{dbl})
                AND UAB_DATA_BAIXA = 1"""

    df = db_connections.oracle2pd('ecap', q)

    return df

def ecap_ui(dbl):
    
    q = f"""SELECT 
    		UNI_CODI_UP AS UP,
    		UNI_CODI_UNITAT AS UI,
    		UNI_DESCRIPCIO AS INFERMERA,
            '{dbl}' AS ETL_SOURCE,
            '{datetime.now()}' AS ETL_TIME
    	FROM 
    		VISTB059{dbl}
    	WHERE 
    		UNI_CODI_UP IN (SELECT USUA_UAB_UP FROM USUTB040{dbl})
            AND UNI_DATA_BAIXA IS NULL"""

    df = db_connections.oracle2pd('ecap', q)

    return df

def ecap_pccmaca(dbl):

    q = f"""SELECT 
                UP_COD_U AS CIP,
                CASE 
                    WHEN UP_COD_PG_SP = 'A0017' THEN 'PCC'
                    WHEN UP_COD_PG_SP = 'A0018' THEN 'MACA'
                END AS DESC_PACIENT,
            	UP_COM,
            	TO_CHAR(UP_DATA_INI, 'DD/MM/YYYY') AS INI_PCC_MACA,
            	TO_CHAR(UP_DATA_FI, 'DD/MM/YYYY') AS FI_PCC_MACA,
                '{dbl}' AS ETL_SOURCE,
                '{datetime.now()}' AS ETL_TIME
        	FROM 
        		PRSTB014{dbl} 
        	WHERE
                UP_COD_U IN (SELECT USUA_CIP FROM USUTB040{dbl})
                AND UP_COD_PG_SP IN ('A0017', 'A0018')
                AND UP_DATA_FI > CURRENT_DATE
                AND UP_DATA_BAIXA IS NULL"""

    df = db_connections.oracle2pd('ecap', q)

    return df

def ecap_up(dbl):

    q = f"""SELECT 
                GCCTB007.UP_CODI_UP_ICS AS UP_ICS,
                GCCTB007.UP_DESC_UP_ICS AS EAP,
                GCCTB008.UP_CODI_UP_SCS AS UP,
                '{dbl}' AS ETL_SOURCE,
                '{datetime.now()}' AS ETL_TIME
        	FROM 
        		GCCTB008{dbl} 
        	LEFT JOIN
                GCCTB007{dbl}
        	ON 
                GCCTB008.UP_CODI_UP_ICS = GCCTB007.UP_CODI_UP_ICS
        	WHERE 
                GCCTB008.UP_CODI_UP_SCS IN (SELECT USUA_UAB_UP FROM USUTB040{dbl})
                AND GCCTB007.UP_DATA_BAIXA IS NULL 
                AND GCCTB008.UP_DATA_BAIXA IS NULL"""

    df = db_connections.oracle2pd('ecap', q)

    return df

In [6]:
%%time

dblinks = ['@p6209_prod','@p6211_prod']

patients = pd.DataFrame()
uabs = pd.DataFrame()
uis = pd.DataFrame()
ups = pd.DataFrame()
pccmaca = pd.DataFrame()

for dblink in dblinks:

    patients = pd.concat([patients, ecap_pat(dblink)])
    uabs = pd.concat([uabs, ecap_uab(dblink)])
    uis = pd.concat([uis, ecap_ui(dblink)])
    ups = pd.concat([ups, ecap_up(dblink)])
    pccmaca = pd.concat([pccmaca, ecap_pccmaca(dblink)])

    print('dblink', dblink, 'extracted!')

patients = patients.sort_values(['CIP','USUA_DATA_ACT']).drop_duplicates('CIP', keep='last').reset_index(drop=True)
patients['UI_SK'] = patients['UP'].astype(str)+'_'+patients['UI']
patients['UAB_SK'] = patients['UP'].astype(str)+'_'+patients['UAB']
patients.drop({'UI','UAB'}, axis=1, inplace=True)

uabs['UAB_SK'] = uabs['UP'].astype(str)+'_'+uabs['UAB']
uabs = uabs.drop_duplicates('UAB_SK', keep='last').reset_index(drop=True)
uabs.drop({'UP','UAB'}, axis=1, inplace=True)

uis['UI_SK'] = uis['UP'].astype(str)+'_'+uis['UI']
uis = uis.drop_duplicates('UI_SK', keep='last').reset_index(drop=True)
uis.drop({'UP','UI'}, axis=1, inplace=True)

pccmaca = pccmaca.sort_values(['CIP','DESC_PACIENT']).drop_duplicates('CIP', keep='first').reset_index(drop=True)

dblink @p6209_prod extracted!
dblink @p6211_prod extracted!
CPU times: user 6.24 s, sys: 1.05 s, total: 7.3 s
Wall time: 2min 2s


In [9]:
# ups.head()

#### pacients

In [ ]:
q = f"""SELECT 
                USUA_CIP AS CIP,
        		TO_DATE(USUA_DATA_NAIXEMENT,'J') AS DATA_NAIXEMENT,
                TO_DATE(USUA_DATA_ACT,'J') AS USUA_DATA_ACT,
        		USUA_SEXE AS SEXE,
        		USUA_UAB_UP AS UP,
        		USUA_UAB_CODI_UAB AS UAB,
        		USUA_CODI_INF AS UI,
        		SUBSTR(GMA_COD, 3, 1) AS GMA,
                GMA_PLNG
        	FROM 
        		USUTB040{dbl}
            LEFT JOIN
                GMATB002{dbl}
            ON
                USUTB040.USUA_CIP=GMATB002.GMA_CIP
        	WHERE 
        		USUTB040.USUA_SITUACIO = 'A'
        		AND USUTB040.USUA_UAB_UP IS NOT NULL
        		AND USUTB040.USUA_DATA_BAIXA = 1"""

# Un cop feta la query s'ha d'eliminar els CIPs duplicats agafant el de ultima usua_data_act

#### uabs

In [ ]:
q = f"""SELECT 
    		UAB_CODI_UP AS UP,
    		UAB_CODI_UAB AS UAB,
    		UAB_DESCRIPCIO AS METGE
    	FROM 
    		VISTB039{dbl}
    	WHERE 
    		UAB_DATA_BAIXA = 1"""

#### uis

In [40]:
q = f"""SELECT 
		UNI_CODI_UP AS UP,
		UNI_CODI_UNITAT AS UI,
		UNI_DESCRIPCIO AS INFERMERA
	FROM 
		VISTB059{dbl}
	WHERE 
		UNI_DATA_BAIXA IS NULL"""

#### eaps

In [ ]:
q = f"""SELECT 
            GCCTB007.UP_CODI_UP_ICS AS UP_ICS,
            GCCTB007.UP_DESC_UP_ICS AS EAP,
            GCCTB008.UP_CODI_UP_SCS AS UP_SCS
    	FROM 
    		GCCTB008{dbl} 
    	LEFT JOIN
            GCCTB007{dbl}
    	ON 
            GCCTB008.UP_CODI_UP_ICS = GCCTB007.UP_CODI_UP_ICS
    	WHERE 
            GCCTB007.UP_DATA_BAIXA IS NULL 
        AND 
            GCCTB008.UP_DATA_BAIXA IS NULL"""

# S'ha de revisar els noms dels descriptius ja que de vegades surten noms amb ABS...

#### pccs / macas

In [ ]:
q = f"""SELECT 
                UP_COD_U AS CIP,
        	CASE 
              WHEN UP_COD_PG_SP = 'A0017' THEN 'PCC'
              WHEN UP_COD_PG_SP = 'A0018' THEN 'MACA'
        	END AS DESC_PACIENT ,
        		UP_COM,
        		TO_CHAR(UP_DATA_INI, 'DD/MM/YYYY') AS INI_PCC_MACA,
        		TO_CHAR(UP_DATA_FI, 'DD/MM/YYYY') AS FI_PCC_MACA  
        	FROM 
        		PRSTB014{dbl}
        	WHERE
                UP_COD_PG_SP IN ('A0017', 'A0018')
            AND
                UP_DATA_FI > CURRENT_DATE
            AND
                UP_DATA_BAIXA IS NULL"""

#### gestoracasos

In [ ]:
q = f"""SELECT
        		GESC_CIP AS CIP,
        		GESC_DATA_ALTA AS GES_CASOS
        	FROM 
        		PRSTB284{dbl}
        	WHERE 
        		GESC_DATA_SORTIDA  IS NULL
        	AND 
                GESC_MOTIU_DENEGAT IS NULL"""

#### prescripcions

In [ ]:
q = f"""SELECT 
        		PPFMC_PMC_USUARI_CIP AS CIP,
        		COUNT(PPFMC_PF_CODI) AS NUM_PRESCRIPCIONS
        	FROM 
                PPFTB016{dbl}
        	WHERE 
                PPFMC_DATA_FI > CURRENT_DATE
        	AND 
                PPFMC_CADUCAT_PER IS NULL 
        	GROUP BY 
        		PPFMC_PMC_USUARI_CIP"""

#### piic

In [ ]:
q = f"""SELECT 
        		MI_CIP AS CIP,
        		MAX(TO_CHAR(MI_DATA_REG,'DD/MM/YYYY')) AS PIIC
        	FROM 
        		PRSTB218{dbl}
        	GROUP BY MI_CIP"""

# afegir where mi_ddb is null

#### resis

In [ ]:
# Per grup

q = f"""SELECT
        		UG_USUA_CIP AS CIP,
        		UG_CODI_GRUP AS RESIDENCIA
        	FROM 
        		PPFTB012{dbl}
            WHERE
                UG_DBA IS NULL"""

#### institucionalitzats

In [88]:
# Resi per diagnòstic

q = f"""SELECT
        		PR_COD_U AS CIP,
        		MAX(PR_DDE) AS INSTITUCIONALITZAT
        	FROM 
        		PRSTB015{dbl}
        	WHERE 
        		PR_COD_PS = 'C01-Z59.3'
            	AND PR_DATA_BAIXA IS NULL 
            	AND PR_DBA IS NULL
            GROUP BY 
        		PR_COD_U"""

#### atdom

In [ ]:
q = f"""SELECT
        		PR_COD_U AS CIP,
        		MAX(PR_DDE) AS ATDOM
        	FROM 
        		PRSTB015{dbl}
        	WHERE 
        		PR_COD_PS = 'C01-Z74.9'
            	AND PR_DATA_BAIXA IS NULL 
            	AND PR_DBA IS NULL
        	GROUP BY 
        		PR_COD_U"""

#### moviments per primària